In [1]:
import pandas as pd
import requests
from pypdf import PdfReader
from io import BytesIO         
import tabula        
import re
import json
import extracao


c:\Users\irani\Desktop\Insper Metricis\Plataforma Metricis\Dados\Projeto-TLC\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_pdfs_validos = pd.read_excel("../Base 5.1.xlsx", sheet_name="PDFs_validos")
df_pdfs_validos

,Nome,Núcleo,Trabalho,Data Publicação,Categoria,Categoria geal,Autores,PDF,Tipo de Arquivo/Link,Resumo
0,Sérgio Lazzarini,Metricis,https://www.scielo.br/j/rae/a/HCdY5w5zcp57GYkR...,2017,Artigo Acadêmico,Acadêmico,sergio giovanetti lazzarini,https://www.scielo.br/j/rae/a/HCdY5w5zcp57GYkR...,PDF,False
1,Leticia Faria de Carvalho Nunes,Saúde,https://repositorio.insper.edu.br/entities/pub...,2021,Working Paper,Pesquisa,leticia faria de carvalho nunes,https://repositorio.insper.edu.br/bitstreams/5...,text/html; charset=utf-8,True
2,Adriana Bruscato Bortoluzzo,Dados,https://repositorio.insper.edu.br/entities/pub...,2020,Artigo Acadêmico,Acadêmico,"adriana bruscato bortoluzzo, silva neto, anton...",https://repositorio.insper.edu.br/bitstreams/5...,text/html; charset=utf-8,True
3,Adriano Dutra Teixeira,Saúde,https://repositorio.insper.edu.br/entities/pub...,2023,Artigo Acadêmico,Acadêmico,Ok,https://repositorio.insper.edu.br/bitstreams/6...,text/html; charset=utf-8,True
4,Andrea Minardi,Metricis,https://repositorio.insper.edu.br/entities/pub...,2013,Working Paper,Pesquisa,ANDREA MARIA ACCIOLY FONSECA MINARDI; Rossi Jú...,https://repositorio.insper.edu.br/bitstreams/6...,text/html; charset=utf-8,True
...,...,...,...,...,...,...,...,...,...,...
517,Jefferson Oliveira de Moura,Pessoas do Poder Público,https://repositorio.insper.edu.br/entities/pub...,2024,Dissertação,Teses e Dissertações,Ok,https://repositorio.insper.edu.br/bitstreams/d...,text/html; charset=utf-8,True
518,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2020,Relatório de pesquisa,Pesquisa,NaN,https://repositorio-api.insper.edu.br/server/a...,PDF,False
519,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2023,Relatório de pesquisa,Pesquisa,NaN,https://repositorio-api.insper.edu.br/server/a...,PDF,False
520,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2022,Relatório de pesquisa,Pesquisa,NaN,https://repositorio-api.insper.edu.br/server/a...,PDF,False


In [3]:
df_pdfs_validos["PDF"]

0      https://www.scielo.br/j/rae/a/HCdY5w5zcp57GYkR...
1      https://repositorio.insper.edu.br/bitstreams/5...
2      https://repositorio.insper.edu.br/bitstreams/5...
3      https://repositorio.insper.edu.br/bitstreams/6...
4      https://repositorio.insper.edu.br/bitstreams/6...
                             ...                        
517    https://repositorio.insper.edu.br/bitstreams/d...
518    https://repositorio-api.insper.edu.br/server/a...
519    https://repositorio-api.insper.edu.br/server/a...
520    https://repositorio-api.insper.edu.br/server/a...
521    https://repositorio-api.insper.edu.br/server/a...
Name: PDF, Length: 522, dtype: object

In [4]:
lista_extraidos = []

for url in df_pdfs_validos["PDF"]:

    try:
        response = requests.get(url)
        response.raise_for_status() 

        pdf_file = BytesIO(response.content)
        reader = PdfReader(pdf_file)

        texto = ""
        if len(reader.pages) >= 15:
            for i in range(0,15):
                page = reader.pages[i]
                texto_formatado = "".join(page.extract_text())
                texto += texto_formatado
            resultado = extracao.extracao_langchain(texto)
            lista_extraidos.append(resultado)
            
        else: 
            for i in range(len(reader.pages)):
                page = reader.pages[i]
                texto_formatado = "".join(page.extract_text())
                texto += texto_formatado
            resultado = extracao.extracao_langchain(texto)
            lista_extraidos.append(resultado)

    except:
        erro = {
        "classificacao": None,
        "metodologia": None,
        "area_avaliada": None,
        "ods": None,
        "etapa": None,
        "titulo": None
        }
        lista_extraidos.append(erro)


lista_extraidos


Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 48 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 59 0 (offset 0)
Ignoring wrong pointing object 61 0 (offset 0)
Ignoring wrong pointing object 63 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 74 0 (offset 0)
Ignoring wrong pointing object 81 0 (offset 0)
Ignoring wrong pointing object 83 0 (offset 0)
Ignoring wrong pointing object 85 0 (offset 0)
Ignoring wrong pointing object 87 0 (offset 0)
Ignoring wrong pointing object 94 0 (offset 0)
Ignoring wrong pointing object 96 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 105 0 (offset 0)
Ignoring wr

[{'classificacao': 'Acadêmica',
  'metodologia': 'Qualitativa',
  'area_avaliada': 'Desenvolvimento Social',
  'ods': '8; 9; 10',
  'etapa': 'Solução; Justificativa',
  'titulo': 'PESQUISA EM ADMINISTRAÇÃO: EM BUSCA DE IMPACTO SOCIAL E OUTROS IMPACTOS'},
 {'classificacao': 'Técnica',
  'metodologia': 'Quantitativa',
  'area_avaliada': 'Saúde',
  'ods': '3',
  'etapa': 'Definição e Dimensão',
  'titulo': 'Panorama da Cobertura Vacinal no Brasil, 2020'},
 {'classificacao': 'Acadêmica',
  'metodologia': 'Quantitativa',
  'area_avaliada': 'Desenvolvimento Social',
  'ods': '8; 9',
  'etapa': 'Certificação',
  'titulo': 'DIVERSIFICATION AND PROPERTY CONTROL IMPACT ON THE PERFORMANCE OF BRAZILIAN REITS'},
 {'classificacao': 'Acadêmica',
  'metodologia': 'Quantitativa',
  'area_avaliada': 'Saúde',
  'ods': '3',
  'etapa': 'Justificativa',
  'titulo': 'Is primary health care worth it in the long run? Evidence from Brazil'},
 {'classificacao': 'Acadêmica',
  'metodologia': 'Qualitativa',
  'are

In [5]:
new_df = pd.DataFrame(lista_extraidos)
new_df

,classificacao,metodologia,area_avaliada,ods,etapa,titulo
0,Acadêmica,Qualitativa,Desenvolvimento Social,8; 9; 10,Solução; Justificativa,PESQUISA EM ADMINISTRAÇÃO: EM BUSCA DE IMPACTO...
1,Técnica,Quantitativa,Saúde,3,Definição e Dimensão,"Panorama da Cobertura Vacinal no Brasil, 2020"
2,Acadêmica,Quantitativa,Desenvolvimento Social,8; 9,Certificação,DIVERSIFICATION AND PROPERTY CONTROL IMPACT ON...
3,Acadêmica,Quantitativa,Saúde,3,Justificativa,Is primary health care worth it in the long ru...
4,Acadêmica,Qualitativa,Desenvolvimento Social,8,Mapeamento dos Determinantes; Solução,Relação entre Investidores e Gestores de Fundo...
...,...,...,...,...,...,...
517,Acadêmica,Quantitativa,Desenvolvimento Social,8,Justificativa,TELETRABALHO E SATISFAÇÃO COM O TRABALHO NO SE...
518,Acadêmica,Mista,Desenvolvimento Social,1; 10; 16,Definição e Dimensão; Mapeamento dos Determina...,A judicialização de benefícios previdenciários...
519,Acadêmica,Quantitativa,Saúde,3; 10,Mapeamento dos Determinantes; Definição e Dime...,A judicialização da saúde infantil: um estudo ...
520,Acadêmica,Mista,Saúde,3,Definição e Dimensão; Mapeamento dos Determina...,AVALIAÇÃO E INCORPORAÇÃO DE TECNOLOGIAS NO BRA...


In [6]:
new_df.to_excel(".teste.xlsx")

In [10]:
df_pdfs_validos_clean = df_pdfs_validos.drop(columns=["PDF", "Tipo de Arquivo/Link", "Resumo"])
df_pdfs_validos_clean

,Nome,Núcleo,Trabalho,Data Publicação,Categoria,Categoria geal,Autores
0,Sérgio Lazzarini,Metricis,https://www.scielo.br/j/rae/a/HCdY5w5zcp57GYkR...,2017,Artigo Acadêmico,Acadêmico,sergio giovanetti lazzarini
1,Leticia Faria de Carvalho Nunes,Saúde,https://repositorio.insper.edu.br/entities/pub...,2021,Working Paper,Pesquisa,leticia faria de carvalho nunes
2,Adriana Bruscato Bortoluzzo,Dados,https://repositorio.insper.edu.br/entities/pub...,2020,Artigo Acadêmico,Acadêmico,"adriana bruscato bortoluzzo, silva neto, anton..."
3,Adriano Dutra Teixeira,Saúde,https://repositorio.insper.edu.br/entities/pub...,2023,Artigo Acadêmico,Acadêmico,Ok
4,Andrea Minardi,Metricis,https://repositorio.insper.edu.br/entities/pub...,2013,Working Paper,Pesquisa,ANDREA MARIA ACCIOLY FONSECA MINARDI; Rossi Jú...
...,...,...,...,...,...,...,...
517,Jefferson Oliveira de Moura,Pessoas do Poder Público,https://repositorio.insper.edu.br/entities/pub...,2024,Dissertação,Teses e Dissertações,Ok
518,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2020,Relatório de pesquisa,Pesquisa,NaN
519,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2023,Relatório de pesquisa,Pesquisa,NaN
520,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2022,Relatório de pesquisa,Pesquisa,NaN


In [11]:
df_final = pd.concat([df_pdfs_validos_clean, new_df], axis=1)
df_final

,Nome,Núcleo,Trabalho,Data Publicação,Categoria,Categoria geal,Autores,classificacao,metodologia,area_avaliada,ods,etapa,titulo
0,Sérgio Lazzarini,Metricis,https://www.scielo.br/j/rae/a/HCdY5w5zcp57GYkR...,2017,Artigo Acadêmico,Acadêmico,sergio giovanetti lazzarini,Acadêmica,Qualitativa,Desenvolvimento Social,8; 9; 10,Solução; Justificativa,PESQUISA EM ADMINISTRAÇÃO: EM BUSCA DE IMPACTO...
1,Leticia Faria de Carvalho Nunes,Saúde,https://repositorio.insper.edu.br/entities/pub...,2021,Working Paper,Pesquisa,leticia faria de carvalho nunes,Técnica,Quantitativa,Saúde,3,Definição e Dimensão,"Panorama da Cobertura Vacinal no Brasil, 2020"
2,Adriana Bruscato Bortoluzzo,Dados,https://repositorio.insper.edu.br/entities/pub...,2020,Artigo Acadêmico,Acadêmico,"adriana bruscato bortoluzzo, silva neto, anton...",Acadêmica,Quantitativa,Desenvolvimento Social,8; 9,Certificação,DIVERSIFICATION AND PROPERTY CONTROL IMPACT ON...
3,Adriano Dutra Teixeira,Saúde,https://repositorio.insper.edu.br/entities/pub...,2023,Artigo Acadêmico,Acadêmico,Ok,Acadêmica,Quantitativa,Saúde,3,Justificativa,Is primary health care worth it in the long ru...
4,Andrea Minardi,Metricis,https://repositorio.insper.edu.br/entities/pub...,2013,Working Paper,Pesquisa,ANDREA MARIA ACCIOLY FONSECA MINARDI; Rossi Jú...,Acadêmica,Qualitativa,Desenvolvimento Social,8,Mapeamento dos Determinantes; Solução,Relação entre Investidores e Gestores de Fundo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,Jefferson Oliveira de Moura,Pessoas do Poder Público,https://repositorio.insper.edu.br/entities/pub...,2024,Dissertação,Teses e Dissertações,Ok,Acadêmica,Quantitativa,Desenvolvimento Social,8,Justificativa,TELETRABALHO E SATISFAÇÃO COM O TRABALHO NO SE...
518,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2020,Relatório de pesquisa,Pesquisa,NaN,Acadêmica,Mista,Desenvolvimento Social,1; 10; 16,Definição e Dimensão; Mapeamento dos Determina...,A judicialização de benefícios previdenciários...
519,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2023,Relatório de pesquisa,Pesquisa,NaN,Acadêmica,Quantitativa,Saúde,3; 10,Mapeamento dos Determinantes; Definição e Dime...,A judicialização da saúde infantil: um estudo ...
520,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2022,Relatório de pesquisa,Pesquisa,NaN,Acadêmica,Mista,Saúde,3,Definição e Dimensão; Mapeamento dos Determina...,AVALIAÇÃO E INCORPORAÇÃO DE TECNOLOGIAS NO BRA...


In [16]:
df_final.to_excel("pdfs.xlsx", index=False)